In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
import json
import pickle
import json

from tqdm import tqdm
import itertools

from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_recall_curve, roc_curve, auc
from sklearn.metrics import make_scorer, roc_auc_score

from torchsurv.loss import cox
from lifelines.utils import concordance_index

sys.path.append('./../src/')
from utils import *
from utils_CPHMLP import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset


# Configs

In [2]:
lr_ls = [0.1, 0.001, 0.00001]
hidden_size_ls = [32, 64, 128, 256, 512]
l2_reg_ls = [0.1, 0.01, 0.001]
batch_size = [16, 32, 64]
    

In [3]:
for idx, (lr_, hidden_size_, l2_reg_, batch_size_) in tqdm(enumerate(itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size))):
    with open('./CPHMLP/configs/config_'+str(idx)+'.json', 'w') as f:
        json.dump({'idx':idx,
                     'lr':lr_,
                     'hidden_size':hidden_size_,
                     'l2_reg':l2_reg_,
                     'batch_size':batch_size_}, f)
    f.close()

135it [00:00, 1477.22it/s]


# Hyperparameter tuning result

In [2]:
lr_ls = [0.1, 0.001, 0.00001]
hidden_size_ls = [32, 64, 128, 256, 512]
l2_reg_ls = [0.1, 0.01, 0.001]
batch_size = [16, 32, 64]
combinations = [i for i in itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size)]
ncombinations = len([i for i in itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size)])
ncombinations


135

In [4]:
results_ls = []
for i in range(ncombinations):
    with open('./../CPHMLP/results/result_config_'+str(i)+'.pkl','rb') as f:
        result = pickle.load(f)
    f.close()
    df = pd.DataFrame(result)
    df['idx'] = [i]*3
        results_ls = results_ls + [df.mean().to_frame().T]
    
pd.concat(results_ls).sort_values('c_index_valid')

,seed,learning_rate,hidden_size,l2_reg,batch_size,time,c_index_train,c_index_valid,c_index_test,nepochs,idx
0,349.333333,0.10000,512.0,0.010,32.0,132.313180,0.455505,0.464100,0.420554,86.666667,40.0
0,349.333333,0.10000,256.0,0.010,16.0,65.137644,0.496488,0.496265,0.466802,86.333333,30.0
0,349.333333,0.10000,64.0,0.010,32.0,39.203393,0.595168,0.499680,0.575335,135.333333,13.0
0,349.333333,0.10000,32.0,0.100,16.0,28.386799,0.500000,0.500000,0.500000,129.333333,0.0
0,349.333333,0.10000,512.0,0.100,16.0,210.161044,0.500000,0.500000,0.500000,62.000000,36.0
...,...,...,...,...,...,...,...,...,...,...,...
0,349.333333,0.00001,128.0,0.001,64.0,25.253568,0.924038,0.745311,0.755678,144.666667,116.0
0,349.333333,0.00001,32.0,0.010,64.0,34.013661,0.909237,0.746302,0.759232,218.000000,95.0
0,349.333333,0.00001,32.0,0.001,64.0,74.486024,0.906534,0.748371,0.757333,211.333333,98.0
0,349.333333,0.00001,64.0,0.010,64.0,30.975312,0.910905,0.753469,0.737979,173.000000,104.0
